Model that accurately recognize food items from images and estimate their calorie content

In [5]:
import tensorflow as tf  # Import TensorFlow library for deep learning tasks
import matplotlib.image as img  # Import matplotlib for image reading
import numpy as np  # Import NumPy for numerical operations
from collections import defaultdict  # Import defaultdict for creating dictionaries with default values
import collections  # Import collections module for collection data types
from shutil import copy  # Import shutil for high-level file operations
from shutil import copytree, rmtree  # Import shutil for directory copying and removal
import tensorflow.keras.backend as K  # Import Keras backend functions
from tensorflow.keras.models import load_model  # Import Keras function for loading pre-trained models
from tensorflow.keras.preprocessing import image  # Import Keras for image preprocessing
import matplotlib.pyplot as plt  # Import matplotlib for visualization
import os  # Import os module for operating system functions
import random  # Import random module for generating random numbers
import cv2  # Import OpenCV for image processing
from tensorflow.keras import regularizers  # Import regularizers for regularization techniques
from tensorflow.keras.applications.inception_v3 import InceptionV3  # Import pre-trained InceptionV3 model
from tensorflow.keras.models import Sequential, Model  # Import Sequential and Model for building neural network models
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten  # Import layers for building neural network architectures
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D  # Import layers for building convolutional neural networks
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Import ImageDataGenerator for data augmentation
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger  # Import callbacks for model saving and logging
from tensorflow.keras.optimizers import SGD  # Import SGD optimizer for training models
from tensorflow.keras.regularizers import l2  # Import L2 regularization
from tensorflow import keras  # Import Keras for deep learning tasks
from tensorflow.keras import models  # Import Keras for building neural network models
import zipfile
import os

dataset used(https://www.kaggle.com/datasets/dansbecker/food-101/code)

In [ ]:
# Check TensorFlow version
print(tf._version_)

# Check GPU device name
print(tf.test.gpu_device_name())

In [ ]:
%cd /kaggle/input/food-101/

In [ ]:
# Downloading and extracting the dataset

def get_data_extract():
    """
    Check if the dataset exists, if not, download and extract it.
    """
    if "food-101" in os.listdir():
        print("Dataset exists!")
    else:
        print("Downloading the data...")
        !wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
        print("Dataset downloaded!")
        print("Extracting data..")
        !tar xzvf food-101.tar.gz
        print("Extraction done!")

In [ ]:
get_data_extract()

In [ ]:
os.listdir('food-101/images')

In [ ]:
os.listdir('food-101/meta')

In [ ]:
!head food-101/meta/train.txt

In [ ]:
!head food-101/meta/classes.txt

In [ ]:
rows = 17
cols = 6

# Subplot grid with specified size
fig, ax = plt.subplots(rows, cols, figsize=(25,25))

fig.suptitle("One random image from each class", y=1.05, fontsize=24)

data_dir = "food-101/images/"

foods_sorted = sorted(os.listdir(data_dir))

food_id = 0

for i in range(rows):
    for j in range(cols):
        try:
            food_selected = foods_sorted[food_id] 
            food_id += 1
        except:
            break
        if food_selected == '.DS_Store':
            continue
        # List of images for the current food class
        food_selected_images = os.listdir(os.path.join(data_dir, food_selected))
        # Select a random image from the list
        food_selected_random = np.random.choice(food_selected_images)
        # Read and display the image
        img = plt.imread(os.path.join(data_dir, food_selected, food_selected_random))
        ax[i][j].imshow(img)
        ax[i][j].set_title(food_selected, pad=10)  # Set the title of the subplot


plt.setp(ax, xticks=[], yticks=[])

plt.tight_layout()

In [ ]:
def prepare_data(filepath, src, dest):
    # Store image paths for each class
    classes_images = defaultdict(list)
    
       with open(filepath, 'r') as txt:
        paths = [read.strip() for read in txt.readlines()]
        for p in paths:
            food = p.split('/')
            classes_images[food[0]].append(food[1] + '.jpg')

    # Iterate over classes and copy images to destination folder
    for food in classes_images.keys():
        print("\nCopying images into ", food)
        if not os.path.exists(os.path.join(dest, food)):
            os.makedirs(os.path.join(dest, food))
        for i in classes_images[food]:
            copy(os.path.join(src, food, i), os.path.join(dest, food, i))
    print("Copying Done!")

In [ ]:
# Change current directory to the root directory
%cd /

print("Creating train data...")

# Copy images from train.txt to train directory
prepare_data('/kaggle/input/food-101/food-101/meta/train.txt', '/kaggle/input/food-101/food-101/images', 'train')

In [ ]:
print("Creating test data...")

# Prepare_data function to copy images from test.txt to test directory
prepare_data('/kaggle/input/food-101/food-101/meta/test.txt', '/kaggle/input/food-101/food-101/images', 'test')

In [ ]:
# Create train_mini and test_mini data samples
def dataset_mini(food_list, src, dest):
    # Check if the destination directory exists
    if os.path.exists(dest):
       
        rmtree(dest)  # Removing dataset_mini (if it already exists) folders
    # Create the destination directory
    os.makedirs(dest)
    
    for food_item in food_list:
        print("Copying images into", food_item)
        # Copy the images from the source to the destination directory for each food item
        copytree(os.path.join(src, food_item), os.path.join(dest, food_item))

In [ ]:
food_list = ['apple_pie', 'pizza', 'omelette']

src_train = 'train'
dest_train = 'train_mini'
src_test = 'test'
dest_test = 'test_mini'

# Train_mini dataset
dataset_mini(food_list, src_train, dest_train)

# Test_mini dataset
dataset_mini(food_list, src_test, dest_test)

In [ ]:
print("Creating train data folder with new classes")

dataset_mini(food_list, src_train, dest_train)

In [ ]:
print("Creating test data folder with new classes")
dataset_mini(food_list, src_test, dest_test)

In [ ]:
def plot_accuracy(history, title):

 plt.title(title)
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train_accuracy', 'validation_accuracy'], loc='best')
    plt.show()

def plot_loss(history, title):

    plt.title(title)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train_loss', 'validation_loss'], loc='best')
    plt.show()

In [ ]:
plot_accuracy(history, 'FOOD101-Inceptionv3')  # Training and validation accuracy
plot_loss(history, 'FOOD101-Inceptionv3')      # Training and validation loss

In [ ]:
%%time

K.clear_session()  # Clear Keras session
model_best = load_model('best_model_3class.hdf5', compile=False)  # Load the best saved model

In [ ]:
def predict_class(model, images, show=True):

    for img in images:
        img = image.load_img(img, target_size=(299, 299)) 
        img = image.img_to_array(img)                    
        img = np.expand_dims(img, axis=0)                        
        img /= 255.                                       
        pred = model.predict(img)                         
        index = np.argmax(pred)                           # Get the index of the class with the highest probability
        food_list.sort()                                  # Sort the list of food items
        pred_value = food_list[index]                     # Get the predicted class label
        
        if show:
            plt.imshow(img[0])                          
            plt.axis('off')
            plt.title(pred_value)                        # Set title as the predicted class label
            plt.show()

In [ ]:
def pick_n_random_classes(n):
    food_list = []
    random_food_indices = random.sample(range(len(foods_sorted)), n)  # Sample n random indices
    for i in random_food_indices:
        food_list.append(foods_sorted[i])  # Retrieve corresponding food items
    food_list.sort()  # Sort the list of randomly selected food classes
    return food_list

In [ ]:
n = 11  # Number of random food classes to select
food_list = pick_n_random_classes(n)  # Select n random food classes
food_list = ['apple_pie', 'beef_carpaccio', 'bibimbap', 'cup_cakes', 'foie_gras', 'french_fries', 'garlic_bread', 'pizza', 'spring_rolls', 'spaghetti_carbonara', 'strawberry_shortcake']
print("These are the randomly picked food classes we will be training the model on...\n", food_list)

In [ ]:
K.clear_session()

n_classes = n

img_width, img_height = 299, 299

train_data_dir = 'train_mini'
validation_data_dir = 'test_mini'

nb_train_samples = 8250
nb_validation_samples = 2750

batch_size = 16

# Define data augmentation for training images
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

# Generate batches of augmented training and validation data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

# Load the InceptionV3 model pretrained on ImageNet without the top layer
inception = InceptionV3(weights='imagenet', include_top=False)

# Add custom top layers for classification
x = inception.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(n, kernel_regularizer=regularizers.l2(0.005), activation='softmax')(x)


model = Model(inputs=inception.input, outputs=predictions)

# Compile the model
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks for model checkpointing and logging
checkpointer = ModelCheckpoint(filepath='best_model_11class.hdf5', verbose=1, save_best_only=True)
csv_logger = CSVLogger('history_11class.log')

# Train the model
history_11class = model.fit_generator(train_generator,
                    steps_per_epoch=nb_train_samples // batch_size,
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size,
                    epochs=30,
                    verbose=1,
                    callbacks=[csv_logger, checkpointer])

# Save the trained model
model.save('model_trained_11class.hdf5')

In [ ]:
class_map_11 = train_generator.class_indices
class_map_11

In [ ]:
images = []
images.append('cupcakes.jpg')
images.append('pizza.jpg')
images.append('springrolls.jpg')
images.append('garlicbread.jpg')
predict_class(model_best, images, True)

In [ ]:
K.clear_session()
print("Loading the model..")
model = load_model('best_model_3class.hdf5', compile=False)
print("Done!")

In [ ]:
model.summary()

In [ ]:
layer_names = []

# Iterate through the layers of the model starting from index 1 and ending at index 10 (inclusive)
for layer in model.layers[1:11]:
    # Append the name of each layer to the list
    layer_names.append(layer.name)

# Print the list of layer names
print(layer_names)

In [ ]:
food = 'applepie.jpg'

# Get the activations of the model for the specified image using the defined activations_output model
activations = get_activations(food, activations_output)

In [ ]:
ind = layer_names.index('activation_1')
sparse_activation = activations[ind]
# Select the activation values of a specific filter
a = sparse_activation[0, :, :, 13]

In [ ]:
all(np.isnan(a[j][k]) for j in range(a.shape[0]) for k in range(a.shape[1]))
#This line checks if all elements in the array a are NaN.

In [ ]:
def get_attribution(food):
    # Load and preprocess the input image
    img = image.load_img(food, target_size=(299, 299))
    img = image.img_to_array(img) 
    img /= 255. 

    # Display the input image
    f, ax = plt.subplots(1, 3, figsize=(15, 15))
    ax[0].imshow(img)
    ax[0].set_title("Input Image")

    # Expand the dimensions and predict the class probabilities
    img = np.expand_dims(img, axis=0) 
    preds = model.predict(img)
    class_id = np.argmax(preds[0])

    # Get the class output and last convolutional layer
    class_output = model.output[:, class_id]
    last_conv_layer = model.get_layer("mixed10")
    # Calculate gradients and pooled gradients
    grads = K.gradients(class_output, last_conv_layer.output)[0]
    pooled_grads = K.mean(grads, axis=(0, 1, 2))
    iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])
    pooled_grads_value, conv_layer_output_value = iterate([img])

    # Generate heatmap
    for i in range(2048):
        conv_layer_output_value[:, :, i] *= pooled_grads_value[i]
    heatmap = np.mean(conv_layer_output_value, axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)
    ax[1].imshow(heatmap)
    ax[1].set_title("Heat map")

    # Overlay heatmap on the original image
    act_img = cv2.imread(food)
    heatmap = cv2.resize(heatmap, (act_img.shape[1], act_img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed = cv2.addWeighted(act_img, 0.6, heatmap, 0.4, 0)
    cv2.imwrite('classactivation.png', superimposed)
    img_act = image.load_img('classactivation.png', target_size=(299, 299))
    ax[2].imshow(img_act)
    ax[2].set_title("Class Activation")
    plt.show()
    return preds

In [ ]:
print("Showing the class map..")
print(class_map_3)

In [ ]:
food = 'piepizza.jpg'
activations = get_activations(food,activations_output)

In [ ]:
show_activations(activations, layer_names)

In [ ]:
pred = get_attribution('piepizza.jpg')
print("Here are softmax predictions..",pred)

In [ ]:
# Data preprocessing
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(train_dir, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, class_mode='categorical')

In [ ]:
# Define the model architecture
input_layer = Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
x = Conv2D(32, (3, 3), activation='relu')(input_layer)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
output_layer = Dense(len(train_generator.class_indices), activation='softmax')(x)

In [ ]:
model = Model(input_layer, output_layer)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_generator, epochs=EPOCHS, validation_data=test_generator)

In [ ]:
# Visualize model predictions
def visualize_predictions(model, test_generator, num_samples=5):
    class_names = list(test_generator.class_indices.keys())
    sample_images, sample_labels = next(test_generator)
    predicted_labels = model.predict(sample_images)
    predicted_classes = np.argmax(predicted_labels, axis=1)

    plt.figure(figsize=(15, 10))
    for i in range(num_samples):
        plt.subplot(1, num_samples, i+1)
        plt.imshow(sample_images[i])
        plt.title(f"True: {class_names[np.argmax(sample_labels[i])]}, Predicted: {class_names[predicted_classes[i]]}")
        plt.axis('off')
    plt.show()

visualize_predictions(model, test_generator)

In [ ]:
# Calorie Estimation (Placeholder)
def estimate_calories(food_item):
    # Replace with your actual calorie estimation model
    return np.random.randint(50, 500)

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Accuracy:", test_accuracy)

In [ ]:
# Make predictions
y_pred = model.predict(test_generator)
y_pred = np.argmax(y_pred, axis=1)
y_true = test_generator.classes

In [ ]:
# Classification report
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=test_generator.class_indices.keys()))

In [ ]:
# Confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

In [ ]:
model.save("food_recognition_model.h5")